Visualise various quantities from a single semiclassical simulation. Assumes that you saved "data", "quantum", and "bilinears" at the same timesteps, and that you run with fixed rmax.

In [ ]:
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/quantum_runs/"
par = "a0.4_b0.0_c4.0_rmax15.0_tmax7.0_cfl0.0625_sigma0.0_overMp2_25.132741228718345_reg_true_backreact_false_mPV100.0_dk_denom_15_kmax10.0_lmax30.0"
your_dir = dir*par

In [ ]:
# gridpoints
D = 3
Nr = 128*2^D + 3 # the overal course graining

# load the r grid
r = h5read(your_dir*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
(its, all_filenames)           = list_h5_files(your_dir*"/data_$(Nr)", prefix="data_");
(its_bln, all_filenames_bln)   = list_h5_files(your_dir*"/data_$(Nr)", prefix="bilinears_");
(its_quantum, all_filenames_quantum)   = list_h5_files(your_dir*"/data_$(Nr)", prefix="quantum_");

# timesteps when data are saved
println("length(its)=",length(its))
println("its",its)

println("length(its_bln)=",length(its_bln))
println("its_bln=",its_bln)

println("length(its_quantum)=",length(its_quantum))
println("its_quantum=",its_quantum)

In [ ]:
# choose an iteration to visualise
it = 120

it_data = its[it]
it_str_data  = lpad(it_data, 4, "0")

it_bln = its_bln[it]
it_str_bln  = lpad(it_bln, 4, "0")

it_quantum = its_quantum[it]
it_str_quantum  = lpad(it_quantum, 4, "0")

v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
v = h5read(your_dir*"/data_$(Nr)/data_$(it_str_data).h5","v")
Π  = v[:,2]
Ψ  = v[:,3]
A  = v[:,4]
oA = 1.0./A
B  = v[:,5]
oB = 1.0./B
KB = v[:,9]
α  = v[:,11]
K  = v[:,8]

bln  = h5read(your_dir*"/data_$(Nr)/bilinears_$(it_str_bln).h5","bilinears")
v_quantum = h5read(your_dir*"/data_$(Nr)/quantum_$(it_str_quantum).h5","v")

# if there is an error when loading the time attribute, try to change "./" to "." or to "/" 
t_data    =  h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
t_bln     =  h5readattr(your_dir*"/data_$(Nr)/bilinears_$(it_str_bln).h5", "/")["time"]
t_quantum =  h5readattr(your_dir*"/data_$(Nr)/quantum_$(it_str_quantum).h5", "/")["time"]

# checks if time instant is the same for all types of data
@assert t_data ≈ t_bln
@assert t_data ≈ t_quantum

mPV = 1.0
CC = -log(3^9/2^16)*(mPV^4)/(8*(2*π)^2);
id = ones(length(r));

# classical
ρ  = 0.5.*oA.*((oB.^2.0).*(Π.^2.0) .+ Ψ.^2.0)
jA = -(oA.^0.5).*oB.*Π.*Ψ
SA = 0.5.*oA.*((oB.^2.0).*(Π.^2.0) .+ Ψ.^2.0)
SB = 0.5.*oA.*((oB.^2.0).*(Π.^2.0) .- Ψ.^2.0)

# convention
hbar = 1.0
c = 1.0

ρ_quantum = @. (hbar*c^2/(4.0*π))*((0.5/α^2)*bln[:,2] + (0.5/A)*bln[:,3] + (1/B)*bln[:,5] + 0.5*bln[:,1] );
jA_quantum = @. -(hbar*c^2/(4.0*π))*(1.0/α)*bln[:,4];
SA_quantum = @. (hbar*c^2/(4.0*π))*( (0.5/α^2)*bln[:,2] + (0.5/A)*bln[:,3] - (1/B)*bln[:,5] - 0.5*bln[:,1] );
SB_quantum = @. (hbar*c^2/(4.0*π))*( (0.5/α^2)*bln[:,2] -(0.5/A)*bln[:,3] - 0.5*bln[:,1] );

ρ_full = ρ + real.(ρ_quantum)
SA_full = SA + real.(SA_quantum)
SB_full = SB + real.(SB_quantum)
jA_full = jA + real.(jA_quantum)

T_tt = @. α^2*(ρ + real(ρ_quantum));
T_rr = @. A*(SA + real(SA_quantum));
T_θθ = @. B*r^2*(SB + real(SB_quantum));

# matter content (sources) in K rhs = α(0.5*SA + SB - Λ + 0.5*ρ)/Mp^2
source_K = 0.5*SA_full + SB_full - CC*id + 0.5*ρ_full;

# matter content (sources) in KB rhs = 0.5*α(SA - Λ - ρ)/Mp^2
source_KB = SA_full - 2.0*CC*id - ρ_full;

In [ ]:
ri_min = 3
ri_max = 2*512
bln_i = 4
blni = bln[:,bln_i]

plot(r[ri_min:ri_max], real.(blni)[ri_min:ri_max], title = "t=$(t_bln)",
     label = "Re(bln[$(bln_i)])", linewidth = 2, frame = true, wsize = (800,400),
     legend=:bottomright)

In [ ]:
ri_min = 3
ri_max = 512*2
ll = 30.0 # l index
kk = 5.0 # k index
mi = 1 # for mass
fi = 1 # 1 is uq, 2 ψq, 3 πq

f = real.((v_quantum)[:,Int(ll)+1,Int(kk),mi,fi]) 
plot(r[ri_min:ri_max], f[ri_min:ri_max], title = "t=$(t_bln)",
     label = "f", linewidth = 2, frame = true, wsize = (800,400))

In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], (ρ_full)[ri_min:ri_max], title = "t=$(t_bln)",
     label = "ρ_full", linewidth = 2, frame = true, wsize = (800,400),
legend=:top)

In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], (SA_full)[ri_min:ri_max], title = "t=$(t_bln)",
     label = "SA_full", linewidth = 2, frame = true, wsize = (800,400),
legend=:top)

In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], (SB_full)[ri_min:ri_max], title = "t=$(t_bln)",
     label = "SB_full", linewidth = 2, frame = true, wsize = (800,400),
legend=:bottomright)

In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], source_K[ri_min:ri_max], title = "t=$(t_bln)",
     label = "source_K", linewidth = 2, frame = true, wsize = (800,400))

In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], source_KB[ri_min:ri_max], title = "t=$(t_bln)",
     label = "source_KB", linewidth = 2, frame = true, wsize = (800,400))

In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], jA[ri_min:ri_max], title = "t=$(t_bln)",
     label = "jA", linewidth = 2, frame = true, wsize = (800,400))

In [ ]:
ri_min = 3
ri_max = 2*512

plot(r[ri_min:ri_max], α[ri_min:ri_max], title = "t=$(t_bln)",
     label = "α", linewidth = 2, frame = true, wsize = (800,400))

In [ ]:
ri_min = 3
ri_max = 2*512

plot(r[ri_min:ri_max], B[ri_min:ri_max], title = "t=$(t_bln)",
     label = "B", linewidth = 2, frame = true, wsize = (800,400))

In [ ]:
ri_min = 3
ri_max = 2*512

plot(r[ri_min:ri_max], A[ri_min:ri_max], title = "t=$(t_bln)",
     label = "A", linewidth = 2, frame = true, wsize = (800,400))

In [ ]:
ri_min = 3
ri_max = 2*512

plot(r[ri_min:ri_max], K[ri_min:ri_max], title = "t=$(t_bln)",
     label = "K", linewidth = 2, frame = true, wsize = (800,400))
plot!([r[end]-t_bln], seriestype="vline", label="r=rmax_causal", linewidth=1, color = "black",legend=:bottomright)


In [ ]:
ri_min = 3
ri_max = 1*512

plot(r[ri_min:ri_max], KB[ri_min:ri_max], title = "t=$(t_bln)",
     label = "KB", linewidth = 2, frame = true, wsize = (800,400))
plot!([r[end]-t_bln], seriestype="vline", label="r=rmax_causal", linewidth=1, color = "black",legend=:bottomleft)
